In [2]:
import pandas as pd
import numpy as np

In [14]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [17]:
import asyncpraw

In [18]:
reddit = await asyncpraw.Reddit(
    client_id=os.environ["REDDIT_CLIENT_ID"],
    client_secret=os.environ["REDDIT_CLIENT_SECRET"],
    # password=os.environ.get("REDDIT_PASSWORD"),
    user_agent=os.environ["USER_AGENT"]
    #, username=os.envirton.get("REDDIT_USERNAME")
).__aenter__()

# reddit.read_only = True|False (if using OAuth, default=False if OAuth is used)

### Ratelimits

In [5]:
print(reddit.auth.limits['used']) # if connected to OAuth

None


In [6]:
print(reddit.auth.limits['remaining']) # if connected to OAuth

None


In [7]:
print(reddit.auth.limits['reset_timestamp']) # if connected to OAuth

None


### Data retrieval

In [19]:
wsb = await reddit.subreddit("wallstreetbets")

In [16]:
submission_df = []

In [17]:
submissions = wsb.new(limit=2) # Must recreate for each async iteration
async for submission in submissions:
    await submission._fetch() # Necessary line
    submission_comments = []
    i = 0
    async for comment in submission.comments:
        if i != 0:
            submission_comments.append(comment.body)
        i += 1
    submission_df.append([submission.title, submission.url, submission.selftext, submission_comments])

In [ ]:
pd.read_csv(os.getenv("COMMENT_CSV"), nrows=1)

,id,author,body,score,created_utc,submission
0,c4b0pvu,secondhandsondek,This is a fantastic idea! I'll toss mine up in...,2,2012-04-11T16:46:43Z,NaN


In [16]:
pd.read_csv(os.getenv("SUBMISSION_CSV"), nrows=1)

,id,author,title,url,selftext,created_utc
0,s4jw1,[deleted],Earnings season is here. Place your bets.,http://www.reddit.com/r/wallstreetbets/comment...,I know that /r/investing is a great place for ...,2012-04-11T16:40:40Z


In [18]:
pd.DataFrame(submission_df)

,0,1,2,3
0,Insider crypto hyper wallet trade,https://www.reddit.com/gallery/1ou26qf,🚨 Hyperliquid wallet 0xb317d2bc2d3d2df5fa441b5...,[]
1,5600% Gains with Options,https://www.reddit.com/r/wallstreetbets/commen...,Been investing with bi-weekly every time i rec...,"[Good job, now buy SPY with 80% and do it agai..."


In [89]:
await reddit.close()

In [ ]:
subreddit = await reddit.subreddit("wallstreetbets+finance+investing")
async for submission in subreddit.sticky():
    print(submission)

In [81]:
subreddit = await reddit.subreddit("wallstreetbets")
thread = await subreddit.sticky(2) # What are your Moves tomorrow

In [ ]:
await thread.comments.replace_more(limit=None)

In [ ]:
comments = thread.comments.list()

In [ ]:
sticky_df = []
subreddit = await reddit.subreddit("wallstreetbets")
thread = await subreddit.sticky(2)
thread_comments = []
i = 0
async for comment in thread.comments:
    
    if i != 0:
        thread_comments.append(comment.children)
    i += 1
submission_df.append([thread.title, thread.url, thread.selftext, thread_comments])

AttributeError: 'Comment' object has no attribute 'children'

In [ ]:
from collections import deque
from asyncpraw.models import Comment, MoreComments
import asyncio, time

async def harvest_submission(submission, more_budget=75, max_comments=2000, calls_per_min=90):
    # Prune without fetching deeply
    await submission.comments.replace_more(limit=0)

    q = deque(submission.comments)
    comments, expanded, calls, window_start = [], 0, 0, time.time()

    async def rate_gate():
        nonlocal calls, window_start
        calls += 1
        elapsed = time.time() - window_start
        if calls >= calls_per_min and elapsed < 60:
            await asyncio.sleep(60 - elapsed)
            calls, window_start = 0, time.time()
        elif elapsed >= 60:
            calls, window_start = 0, time.time()

    while q and len(comments) < max_comments:
        node = q.popleft()

        if isinstance(node, Comment):
            body = getattr(node, "body", None)
            if body:
                comments.append(body)
            # replies already present are cheap to traverse
            if node.replies:
                q.extend(node.replies)

        elif isinstance(node, MoreComments):
            if expanded >= more_budget:
                continue
            await rate_gate()                      # respect API pacing
            children = await node.comments()       # API call(s)
            expanded += 1
            q.extend(children)

    return comments, expanded
